In [25]:
import xscen as xs
from xscen.config import CONFIG
import figanos.matplotlib as fg
import xarray as xr
import pandas as pd
import numpy as np
import geopandas as gpd
import IPython.display as display

# === Load config and style ===
fg.utils.set_mpl_style('ouranos')
xs.load_config("../paths_obs.yml", "../config_obs.yml", verbose=(__name__ == "__main__"), reset=True)

# === Load project catalog and region shapefile ===
pcat = xs.ProjectCatalog(CONFIG["paths"]["project_catalog"])
pcat_df = pcat.df

gdf = gpd.read_file("../" + CONFIG["regional_mean"]["region"]["shape"])
gdf["centroid_y"] = gdf.geometry.centroid.y
REGION_ORDER = gdf.sort_values("centroid_y")["name"].tolist()
all_regions = REGION_ORDER

# === Get all unique variables from coherence datasets ===
pcat_df_perf = pcat_df[pcat_df["processing_level"] == "coherence"]
variable_groups = pcat_df_perf["variable"].unique()
variables = sorted(set(item for group in variable_groups for item in group))

# === Constants ===
HORIZON = "1981-2010"
SEASON_ORDER = ["DJF", "MAM", "JJA", "SON", "ANN"]


# === Function to generate the table ===
def generate_table_per_region(horizon=HORIZON):
    data_cells = {}

    for spatial_var in variables:
        print(f"Processing variable: {spatial_var}")

        dts = pcat.search(processing_level="coherence", variable=spatial_var).to_dataset_dict()
        if not dts:
            print(f"  ✖ No dataset found for variable '{spatial_var}'")
            continue

        ds = list(dts.values())[0]

        if spatial_var not in ds:
            print(f"  ✖ Variable '{spatial_var}' not found in dataset.")
            continue
        if "region" not in ds.dims or "horizon" not in ds.dims:
            print(f"  ✖ Variable '{spatial_var}' missing 'region' or 'horizon'.")
            continue
        if horizon not in ds["horizon"].values:
            print(f"  ✖ Horizon '{horizon}' not found for '{spatial_var}'")
            continue

        da = ds[spatial_var].sel(horizon=horizon)
        regions = REGION_ORDER

        if "season" in da.dims:
            seasons = [str(s.item()) if hasattr(s, "item") else str(s) for s in da["season"].values]
            for season in seasons:
                vals = da.sel(season=season).reindex(region=regions).values
                series = pd.Series(
                    {region: f"{val:.2f}" if pd.notna(val) else ""
                     for region, val in zip(regions, vals)}
                ).reindex(all_regions, fill_value="")
                data_cells[(spatial_var, season)] = series
        else:
            # Annual-only data (no season dim)
            vals = da.reindex(region=regions).values
            series = pd.Series(
                {region: f"{val:.2f}" if pd.notna(val) else ""
                 for region, val in zip(regions, vals)}
            ).reindex(all_regions, fill_value="")
            data_cells[(spatial_var, "ANN")] = series

    if not data_cells:
        print("No data to display.")
        return

    # === Sort columns by variable, then by fixed season order ===
    season_order_map = {s: i for i, s in enumerate(SEASON_ORDER)}
    sorted_keys = sorted(data_cells.keys(), key=lambda x: (x[0], season_order_map.get(x[1], 99)))
    columns = pd.MultiIndex.from_tuples(sorted_keys, names=["Variable", "Season"])
    df_values = pd.DataFrame(data_cells, index=all_regions, columns=columns).fillna("")
    df_values.index.name = "Region"

    # === Add left border between variables ===
    column_borders = pd.DataFrame("", index=df_values.index, columns=df_values.columns)
    last_var = None
    for col in df_values.columns:
        var = col[0]
        if var != last_var and last_var is not None:
            column_borders[col] = "border-left: 3px solid black;"
        last_var = var

    from matplotlib import cm
    from matplotlib.colors import Normalize, to_hex

    # === Create value-based background color gradient (green to red) ===
    style_data = pd.DataFrame(index=df_values.index, columns=df_values.columns)

    for col in df_values.columns:
        vals = pd.to_numeric(df_values[col], errors='coerce')
        if vals.notna().sum() == 0:
            norm = None
        else:
            norm = Normalize(vmin=vals.min(), vmax=vals.max())

        for idx in df_values.index:
            val = vals.at[idx]
            border = column_borders.at[idx, col]

            if pd.isna(val):
                style = "background-color: black; color: white; text-align: center"
            else:
                rgba = cm.get_cmap("RdYlGn_r")(norm(val))  # green = low, red = high
                hex_color = to_hex(rgba)
                style = f"background-color: {hex_color}; color: black; text-align: center"

            style_data.at[idx, col] = f"{style} {border}"

    # === Apply styles ===
    styled = (
        df_values.style
        .apply(lambda _: style_data, axis=None)
        .set_properties(**{"text-align": "center"})
        .set_table_styles([
            {"selector": "td", "props": [("border", "1px solid #999")]},
            {"selector": "th", "props": [("border", "1px solid #666"), ("text-align", "center")]}
        ])
    )

    display.display(styled)
    print("Regional table generated successfully.")


# === Run the function ===
generate_table_per_region()


2025-07-31 15:54:51 INFO     xscen.config    Updated the config with ../paths_obs.yml.
2025-07-31 15:54:51 INFO     xscen.config    Updated the config with ../config_obs.yml.


Processing variable: pr_mean_annual_clim_mean

--> The keys in the returned dictionary of datasets are constructed as follows:
	'id.processing_level.xrfreq'


/tmp/ipykernel_122434/613764373.py:19: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.



Processing variable: pr_mean_seasonal_clim_mean

--> The keys in the returned dictionary of datasets are constructed as follows:
	'id.processing_level.xrfreq'


Processing variable: tg_mean_annual_clim_mean

--> The keys in the returned dictionary of datasets are constructed as follows:
	'id.processing_level.xrfreq'


Processing variable: tg_mean_seasonal_clim_mean

--> The keys in the returned dictionary of datasets are constructed as follows:
	'id.processing_level.xrfreq'


/tmp/ipykernel_122434/613764373.py:118: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.


Regional table generated successfully.
